In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
from ResDrop152 import ResNet
from chainer import serializers, cuda, Variable
import chainer.functions as F
from progressbar import ProgressBar
import glob
import os

In [2]:
model = ResNet()
model.train = False
serializers.load_hdf5('../results/2016-06-20_12-35-36/model', model)
cuda.check_cuda_available()
cuda.get_device(2).use()
model.to_gpu()

In [3]:
data = glob.glob('../data/test/*')
data.sort()

In [ ]:
pbar = ProgressBar(len(data))
mean = np.load('../data/mean.npy').astype(np.float32)
mean = cv2.resize(mean, (224, 224))
sum_acc = []
bsize = 100
for i in range(0, len(data), bsize):
    x = np.empty((bsize, 3, 224, 224), dtype=np.float32)
    for j in range(0, bsize):
        if i+j == len(data):
            x = x[:j]
            break
        img = cv2.imread(data[i+j]).astype(np.float32)
        img = cv2.resize(img, (224, 224)) - mean
        x[j] = img.transpose(2, 0, 1)
    x = Variable(cuda.cupy.asarray(x), volatile='on')
    h = F.softmax(model(x, None))
    sum_acc += [cuda.to_cpu(h.data),]
    pbar.update(i+1)

  4% (3701 of 79726) |                    | Elapsed Time: 0:00:50 ETA:  0:17:29

In [ ]:
with open('result.csv', 'w') as f:
    f.write('img')
    for i in range(10):
        f.write(',c{}'.format(i))
    f.write('\n')
    
    c = 0
    pbar = ProgressBar(len(data))
    for block in sum_acc:
        for line in block:
            f.write(os.path.basename(data[c]))
            for j in line:
                f.write(',{}'.format(j))
            f.write('\n')
            c += 1
            pbar.update(c)